In [1]:
import torch
import time
from script.NeuralNets.Networks import SequentialNN
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier
import gurobipy as grp
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize
from script.NeuralNets.ICNNFactory import ICNNFactory
from threading import Timer
import matplotlib.pyplot as plt
from script import eval

In [2]:
def add_output_constraints(model, nn_layer_out_bounds, label, output_vars, sovler_bound=1e-3):
    out_lb = nn_layer_out_bounds[-1][0]
    out_ub = nn_layer_out_bounds[-1][1]
    difference_lb = out_lb - out_ub[label]
    difference_ub = out_ub - out_lb[label]
    difference_lb = difference_lb.tolist()
    difference_ub = difference_ub.tolist()
    difference_lb.pop(label)
    difference_ub.pop(label)
    min_diff = min(difference_lb)
    max_diff = max(difference_ub)

    difference = model.addVars(9, lb=difference_lb, ub=difference_ub, name="diff_var")
    model.addConstrs((difference[i] == output_vars.tolist()[i] - output_vars.tolist()[label] for i in range(0, label)), name="diff_const0")
    model.addConstrs((difference[i - 1] == output_vars.tolist()[i] - output_vars.tolist()[label] for i in range(label + 1, 10)), name="diff_const1")

    max_var = model.addVar(lb=min_diff, ub=max_diff, name="max_var")
    model.addConstr(max_var == grp.max_(difference))

    if sovler_bound != None:
        model.setParam("BestObjStop", sovler_bound)

    model.update()
    model.setObjective(max_var, grp.GRB.MAXIMIZE)

In [3]:
def get_output_vars_snv_milp(model, output_size, output_layer_index):
    output_vars = []
    for i in range(output_size):
        output_vars.append(model.getVarByName("affine_var{}[{}]".format(output_layer_index, i)))
    output_vars = grp.MVar.fromlist(output_vars)
    return output_vars

In [4]:
def get_output_vars_dhov(model, output_size, output_layer_index):
    output_vars = []
    for i in range(output_size):
        output_vars.append(model.getVarByName("output_layer_[{}]_[{}]".format(output_layer_index, i)))
    output_vars = grp.MVar.fromlist(output_vars)
    return output_vars

In [5]:
def optimize_model(model, output_vars):
    start = time.time()
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL or model.Status == grp.GRB.USER_OBJ_LIMIT:
        for i, var in enumerate(output_vars.tolist()):
            print("var {}: {}".format(i, var.getAttr("x")))
        max_var = model.getVarByName("max_var").getAttr("x")
        if max_var < 0:
            print("property verified with max difference {}".format(max_var))
            return True
        else:
             print("property NOT verified with max difference {}".format(max_var))
             return False

    elif model.Status == grp.GRB.INFEASIBLE:
        print("model infeasible")

        model.computeIIS()
        print("constraint")
        all_constr = model.getConstrs()

        for const in all_constr:
            if const.IISConstr:
                print("{}".format(const))

        print("lower bound")
        all_var = model.getVars()
        for var in all_var:
            if var.IISLB:
                print("{}, lb: {}, ub: {}".format(var, var.getAttr("lb"), var.getAttr("ub")))

        print("upper bound")
        all_var = model.getVars()
        for var in all_var:
            if var.IISUB:
                print("{}, lb: {}, ub: {}".format(var, var.getAttr("lb"), var.getAttr("ub")))

    print("time to optimize: {}".format(time.time() - start))
    return False

In [6]:
transform = Compose([ToTensor(),  Normalize(0.5, 0.5)])
training_data = MNIST(root="../../mnist", train=True, download=True, transform=transform)

nn = SequentialNN([28 * 28 * 1, 256, 256, 256, 256, 10])
nn.load_state_dict(torch.load("../../mnist_fc 4x256.pth", map_location=torch.device('cpu')), strict=False)
parameter_list = list(nn.parameters())
output_size = 10
number_layer = (len(parameter_list) - 2) // 2

eps = 0.026
samples_to_verify_count = 50

SNR

In [7]:
snr_verifier_list = []
not_verified_index_snr = []
sample_index = 0
round_index = 0
while sample_index < samples_to_verify_count:
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    simple_bounds_affine_out, simple_bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])

    if not torch.argmax(nn(torch_image)).item() == label:
        sample_index += 1
        continue
    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_count-1, sample_index))

    overall_time = time.time()
    snv_verifier = SingleNeuronVerifier(nn, torch_image, eps, print_log=False, print_new_bounds=False, optimize_bounds=True)
    snv_verifier.generate_constraints_for_net()
    snr_verifier_list.append(snv_verifier)
    snv_model = snv_verifier.model.copy()
    snv_model.update()
    snv_out_vars = get_output_vars_snv_milp(snv_model, output_size, number_layer * 2)
    add_output_constraints(snv_model, simple_bounds_layer_out, label, snv_out_vars, sovler_bound=None)

    print("start with optimization")
    successful_ver = optimize_model(snv_model, snv_out_vars)
    if not successful_ver:
        not_verified_index_snr.append((sample_index, round_index))
    print("overall time: {}".format(time.time() - overall_time))

    round_index += 1
    sample_index += 1

0/49 | image index 0 ===========================================================================================
Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-12
start with optimization
var 0: -2.350125426438014
var 1: 2.8104705379689747
var 2: 2.0215793109365645
var 3: 11.135935128310992
var 4: -10.647695250328898
var 5: 8.714479781586537
var 6: -7.866874091667622
var 7: 0.7696455261452932
var 8: -1.6448872973630273
var 9: 0.012855750542410505
property NOT verified with max difference 2.4214553467244553
overall time: 140.87287831306458
1/49 | image index 1 ===========================================================================================
start with optimization
var 0: 9.17468428508412
var 1: -4.544084025518291
var 2: 6.12551216165531
var 3: -0.440940501936927
var 4: -7.6650764172566195
var 5: -0.4662126458829601
var 6: -0.508567563866702
var 7: 1.4868265333823096
var 8: -1.0565707397087305
var 9: -1.7659124202426728
property verified 

DHOV

In [8]:
print(not_verified_index_snr)

[(0, 0), (2, 2), (4, 4), (14, 14), (18, 18), (19, 19), (22, 22), (25, 24), (26, 25), (29, 28), (30, 29), (32, 31), (33, 32), (38, 37), (42, 41), (43, 42), (46, 45), (48, 47)]


In [15]:
group_size = 30
sample_count = 100
sampling_method = "per_group_sampling"
net_size = [5, 1]
icnn_factory = ICNNFactory("logical", net_size, always_use_logical_layer=False)
#icnn_factory = ICNNFactory("standard", net_size, adapt_layer_for_init=True)
#icnn_factory = ICNNFactory("approx_max", net_size, maximum_function="SMU", function_parameter=0.3)

dhov_verifier_list = []
not_verified_index_dhov = []
sample_index = 0
round_index = 0
samples_to_verify_dhov = len(not_verified_index_snr)
for touple in not_verified_index_snr:
    sample_index = touple[0]
    round_index = touple[1]
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    bounds_affine_out_snr, bounds_layer_out_snr = snr_verifier_list[round_index].bounds_affine_out, snr_verifier_list[round_index].bounds_layer_out
    #print(bounds_affine_out_snr)
    #print(bounds_layer_out_snr)
    #bounds_affine_out_snr, bounds_layer_out_snr = None, None

    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_dhov-1, sample_index))


    overall_time = time.time()
    dhov_verifier = multidhov.MultiDHOV()
    dhov_verifier.start_verification(nn, torch_image, icnn_factory, group_size, eps=eps, icnn_epochs=200, init_affine_bounds=bounds_affine_out_snr, init_layer_bounds=bounds_layer_out_snr,
                                     icnn_batch_size=10000, sample_count=sample_count, sample_new=True, use_over_approximation=True, break_after=None,
                                     sample_over_input_space=False, sample_over_output_space=True, tighten_bounds=True,
                                     use_fixed_neurons_in_grouping=False, sampling_method=sampling_method, layers_as_snr=[], layers_as_milp=[3],
                                     force_inclusion_steps=3, preemptive_stop=True, even_gradient_training=False,
                                     keep_ambient_space=False, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=False, print_new_bounds=False,
                                     grouping_method="consecutive", group_num_multiplier=5, store_samples=False, print_optimization_steps=False, print_last_loss=False,
                                     should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="included")


    dhov_verifier_list.append(dhov_verifier)

    dhov_model = dhov_verifier.nn_encoding_model.copy()
    dhov_model.update()
    dhov_out_vars = get_output_vars_dhov(dhov_model, output_size, number_layer)
    add_output_constraints(dhov_model, dhov_verifier.bounds_layer_out, label, dhov_out_vars, sovler_bound=float("inf"))
    successful_ver = optimize_model(dhov_model, dhov_out_vars)
    if not successful_ver:
        not_verified_index_dhov.append(sample_index)
    print("overall time: {}".format(time.time() - overall_time))
    round_index += 1
    sample_index += 1

0/17 | image index 0 ===========================================================================================

approximation of layer: 0
    number of fixed neurons for current layer: 182
    layer progress, group 1 of 3 
        time for sampling for one group: 0.01562666893005371
        time for training: 0.9470484256744385
        actual verification time 17.581485748291016
        time for verification: 17.78248381614685
    layer progress, group 2 of 3 
        time for sampling for one group: 0.015626907348632812
        time for training: 0.5981109142303467
        actual verification time 13.973247051239014
        time for verification: 14.193633794784546
    layer progress, group 3 of 3 
        time for sampling for one group: 0.015640735626220703
        time for training: 0.3605184555053711
        actual verification time 0.5935606956481934
        time for verification: 0.6841051578521729

approximation of layer: 1
    time for icnn_bound calculation: 18.940300464630

In [18]:
group_size = 20
sample_count = 100
sampling_method = "per_group_sampling"
net_size = [5, 1]
icnn_factory = ICNNFactory("logical", net_size, always_use_logical_layer=False)
#icnn_factory = ICNNFactory("standard", net_size, adapt_layer_for_init=True)
#icnn_factory = ICNNFactory("approx_max", net_size, maximum_function="SMU", function_parameter=0.3)

dhov_verifier_list = []
not_verified_index_dhov = []
sample_index = 0
round_index = 0
samples_to_verify_dhov = len(not_verified_index_snr)
for touple in not_verified_index_snr:
    sample_index = touple[0]
    round_index = touple[1]
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    #bounds_affine_out_snr, bounds_layer_out_snr = snr_verifier_list[round_index].bounds_affine_out, snr_verifier_list[round_index].bounds_layer_out
    #print(bounds_affine_out_snr)
    #print(bounds_layer_out_snr)
    bounds_affine_out_snr, bounds_layer_out_snr = None, None

    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_dhov-1, sample_index))


    overall_time = time.time()
    dhov_verifier = multidhov.MultiDHOV()
    dhov_verifier.start_verification(nn, torch_image, icnn_factory, group_size, eps=eps, icnn_epochs=200, init_affine_bounds=bounds_affine_out_snr, init_layer_bounds=bounds_layer_out_snr,
                                     icnn_batch_size=10000, sample_count=sample_count, sample_new=True, use_over_approximation=True, break_after=None,
                                     sample_over_input_space=False, sample_over_output_space=True, tighten_bounds=True,
                                     use_fixed_neurons_in_grouping=False, sampling_method=sampling_method, layers_as_snr=[0,1,2], layers_as_milp=[3],
                                     force_inclusion_steps=3, preemptive_stop=True, even_gradient_training=False,
                                     keep_ambient_space=False, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=False, print_new_bounds=False,
                                     grouping_method="random", group_num_multiplier=3, store_samples=False, print_optimization_steps=False, print_last_loss=False,
                                     should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="included")


    dhov_verifier_list.append(dhov_verifier)

    dhov_model = dhov_verifier.nn_encoding_model.copy()
    dhov_model.update()
    dhov_out_vars = get_output_vars_dhov(dhov_model, output_size, number_layer)
    add_output_constraints(dhov_model, dhov_verifier.bounds_layer_out, label, dhov_out_vars, sovler_bound=float("inf"))
    successful_ver = optimize_model(dhov_model, dhov_out_vars)
    if not successful_ver:
        not_verified_index_dhov.append(sample_index)
    print("overall time: {}".format(time.time() - overall_time))
    round_index += 1
    sample_index += 1

0/17 | image index 0 ===========================================================================================

approximation of layer: 0
    number of fixed neurons for current layer: 182
    encode layer 0 as SNR

approximation of layer: 1
    time for icnn_bound calculation: 12.619257926940918
    number of fixed neurons for current layer: 191
    encode layer 1 as SNR

approximation of layer: 2
    time for icnn_bound calculation: 36.65613794326782
    number of fixed neurons for current layer: 200
    encode layer 2 as SNR

approximation of layer: 3
    time for icnn_bound calculation: 90.70272922515869
    number of fixed neurons for current layer: 192
    encode layer 3 as MILP
time for icnn_bound calculation (last layer):0.010822772979736328
done...
var 0: -2.432210144042422
var 1: 2.7862211076402694
var 2: 1.9096281931308834
var 3: 11.199393627925536
var 4: -10.75471489540369
var 5: 9.031106478451246
var 6: -7.95017792712018
var 7: 0.6621138233454236
var 8: -1.61219819518630

In [19]:
group_size = 20
sample_count = 50
sampling_method = "per_group_sampling"
net_size = [5, 1]
icnn_factory = ICNNFactory("logical", net_size, always_use_logical_layer=False)
#icnn_factory = ICNNFactory("standard", net_size, adapt_layer_for_init=True)
#icnn_factory = ICNNFactory("approx_max", net_size, maximum_function="SMU", function_parameter=0.3)

dhov_verifier_list = []
not_verified_index_dhov = []
sample_index = 0
round_index = 0
samples_to_verify_dhov = len(not_verified_index_snr)
for touple in not_verified_index_snr:
    sample_index = touple[0]
    round_index = touple[1]
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    bounds_affine_out_snr, bounds_layer_out_snr = snr_verifier_list[round_index].bounds_affine_out, snr_verifier_list[round_index].bounds_layer_out
    #print(bounds_affine_out_snr)
    #print(bounds_layer_out_snr)
    #bounds_affine_out_snr, bounds_layer_out_snr = None, None

    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_dhov-1, sample_index))


    overall_time = time.time()
    dhov_verifier = multidhov.MultiDHOV()
    dhov_verifier.start_verification(nn, torch_image, icnn_factory, group_size, eps=eps, icnn_epochs=200, init_affine_bounds=bounds_affine_out_snr, init_layer_bounds=bounds_layer_out_snr,
                                     icnn_batch_size=10000, sample_count=sample_count, sample_new=True, use_over_approximation=True, break_after=None,
                                     sample_over_input_space=False, sample_over_output_space=True, tighten_bounds=True,
                                     use_fixed_neurons_in_grouping=False, sampling_method=sampling_method, layers_as_snr=[], layers_as_milp=[3],
                                     force_inclusion_steps=3, preemptive_stop=True, even_gradient_training=False,
                                     keep_ambient_space=False, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=False, print_new_bounds=False,
                                     grouping_method="consecutive", group_num_multiplier=5, store_samples=False, print_optimization_steps=False, print_last_loss=False,
                                     should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="included")


    dhov_verifier_list.append(dhov_verifier)

    dhov_model = dhov_verifier.nn_encoding_model.copy()
    dhov_model.update()
    dhov_out_vars = get_output_vars_dhov(dhov_model, output_size, number_layer)
    add_output_constraints(dhov_model, dhov_verifier.bounds_layer_out, label, dhov_out_vars, sovler_bound=float("inf"))
    successful_ver = optimize_model(dhov_model, dhov_out_vars)
    if not successful_ver:
        not_verified_index_dhov.append(sample_index)
    print("overall time: {}".format(time.time() - overall_time))
    round_index += 1
    sample_index += 1

0/17 | image index 0 ===========================================================================================

approximation of layer: 0
    number of fixed neurons for current layer: 182
    layer progress, group 1 of 4 
        time for sampling for one group: 0.0


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\DHOV\MultiDHOV.py:95: UserWarning: value for group number multiplier is given with grouping method consecutive. consecutive grouping does not use variable number of groups
  warnings.warn("value for group number multiplier is given with grouping method consecutive. "


        time for training: 0.28950977325439453
        actual verification time 0.47292137145996094
        time for verification: 0.5738933086395264
    layer progress, group 2 of 4 
        time for sampling for one group: 0.0
        time for training: 0.22358465194702148
        actual verification time 1.537365198135376
        time for verification: 1.6480271816253662
    layer progress, group 3 of 4 
        time for sampling for one group: 0.0
        time for training: 0.21160364151000977
        actual verification time 1.7165532112121582
        time for verification: 1.8373112678527832
    layer progress, group 4 of 4 
        time for sampling for one group: 0.0
        time for training: 0.2808248996734619
        actual verification time 0.47865796089172363
        time for verification: 0.580500602722168

approximation of layer: 1
    time for icnn_bound calculation: 21.57001805305481
    number of fixed neurons for current layer: 196
    layer progress, group 1 of 3 
 

In [21]:
group_size = 20
sample_count = 50
sampling_method = "per_group_sampling"
net_size = [5, 1]
icnn_factory = ICNNFactory("logical", net_size, always_use_logical_layer=False)
#icnn_factory = ICNNFactory("standard", net_size, adapt_layer_for_init=True)
#icnn_factory = ICNNFactory("approx_max", net_size, maximum_function="SMU", function_parameter=0.3)

dhov_verifier_list = []
not_verified_index_dhov = []
sample_index = 0
round_index = 0
samples_to_verify_dhov = len(not_verified_index_snr)
for touple in not_verified_index_snr:
    sample_index = touple[0]
    round_index = touple[1]
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    bounds_affine_out_snr, bounds_layer_out_snr = snr_verifier_list[round_index].bounds_affine_out, snr_verifier_list[round_index].bounds_layer_out
    #print(bounds_affine_out_snr)
    #print(bounds_layer_out_snr)
    #bounds_affine_out_snr, bounds_layer_out_snr = None, None

    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_dhov-1, sample_index))


    overall_time = time.time()
    dhov_verifier = multidhov.MultiDHOV()
    dhov_verifier.start_verification(nn, torch_image, icnn_factory, group_size, eps=eps, icnn_epochs=200, init_affine_bounds=bounds_affine_out_snr, init_layer_bounds=bounds_layer_out_snr,
                                     icnn_batch_size=10000, sample_count=sample_count, sample_new=True, use_over_approximation=True, break_after=None,
                                     sample_over_input_space=False, sample_over_output_space=True, tighten_bounds=True,
                                     use_fixed_neurons_in_grouping=False, sampling_method=sampling_method, layers_as_snr=[], layers_as_milp=[3],
                                     force_inclusion_steps=3, preemptive_stop=True, even_gradient_training=False,
                                     keep_ambient_space=False, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=False, print_new_bounds=False,
                                     grouping_method="random", group_num_multiplier=3, store_samples=False, print_optimization_steps=False, print_last_loss=False,
                                     should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="included")


    dhov_verifier_list.append(dhov_verifier)

    dhov_model = dhov_verifier.nn_encoding_model.copy()
    dhov_model.update()
    dhov_out_vars = get_output_vars_dhov(dhov_model, output_size, number_layer)
    add_output_constraints(dhov_model, dhov_verifier.bounds_layer_out, label, dhov_out_vars, sovler_bound=float("inf"))
    successful_ver = optimize_model(dhov_model, dhov_out_vars)
    if not successful_ver:
        not_verified_index_dhov.append(sample_index)
    print("overall time: {}".format(time.time() - overall_time))
    round_index += 1
    sample_index += 1

0/17 | image index 0 ===========================================================================================

approximation of layer: 0
    number of fixed neurons for current layer: 182
    layer progress, group 1 of 12 
        time for sampling for one group: 0.0
        time for training: 0.4007270336151123
        actual verification time 2.2114663124084473
        time for verification: 2.3541557788848877
    layer progress, group 2 of 12 
        time for sampling for one group: 0.0030426979064941406
        time for training: 0.25956225395202637
        actual verification time 0.6036748886108398
        time for verification: 0.7170877456665039
    layer progress, group 3 of 12 
        time for sampling for one group: 0.0
        time for training: 0.267017126083374
        actual verification time 1.7704150676727295
        time for verification: 1.8784663677215576
    layer progress, group 4 of 12 
        time for sampling for one group: 0.0
        time for training: 

In [12]:
group_size = 3
sample_count = 10000
sampling_method = "uniform"
net_size = [5, 1]
icnn_factory = ICNNFactory("logical", net_size, always_use_logical_layer=False)
#icnn_factory = ICNNFactory("standard", net_size, adapt_layer_for_init=True)
#icnn_factory = ICNNFactory("approx_max", net_size, maximum_function="SMU", function_parameter=0.3)

dhov_verifier_list = []
not_verified_index_dhov = []
sample_index = 0
round_index = 0
samples_to_verify_dhov = len(not_verified_index_snr)
for touple in not_verified_index_snr:
    sample_index = touple[0]
    round_index = touple[1]
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    bounds_affine_out_snr, bounds_layer_out_snr = snr_verifier_list[round_index].bounds_affine_out, snr_verifier_list[round_index].bounds_layer_out
    #print(bounds_affine_out_snr)
    #print(bounds_layer_out_snr)
    #bounds_affine_out_snr, bounds_layer_out_snr = None, None

    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_dhov-1, sample_index))


    overall_time = time.time()
    dhov_verifier = multidhov.MultiDHOV()
    dhov_verifier.start_verification(nn, torch_image, icnn_factory, group_size, eps=eps, icnn_epochs=200, init_affine_bounds=bounds_affine_out_snr, init_layer_bounds=bounds_layer_out_snr,
                                     icnn_batch_size=10000, sample_count=sample_count, sample_new=True, use_over_approximation=True, break_after=None,
                                     sample_over_input_space=False, sample_over_output_space=True, tighten_bounds=True,
                                     use_fixed_neurons_in_grouping=False, sampling_method=sampling_method, layers_as_snr=[], layers_as_milp=[3],
                                     force_inclusion_steps=3, preemptive_stop=True, even_gradient_training=False,
                                     keep_ambient_space=False, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=False, print_new_bounds=False,
                                     grouping_method="consecutive", group_num_multiplier=5, store_samples=False, print_optimization_steps=False, print_last_loss=False,
                                     should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="included")


    dhov_verifier_list.append(dhov_verifier)

    dhov_model = dhov_verifier.nn_encoding_model.copy()
    dhov_model.update()
    dhov_out_vars = get_output_vars_dhov(dhov_model, output_size, number_layer)
    add_output_constraints(dhov_model, dhov_verifier.bounds_layer_out, label, dhov_out_vars, sovler_bound=float("inf"))
    successful_ver = optimize_model(dhov_model, dhov_out_vars)
    if not successful_ver:
        not_verified_index_dhov.append(sample_index)
    print("overall time: {}".format(time.time() - overall_time))
    round_index += 1
    sample_index += 1

0/17 | image index 0 ===========================================================================================

approximation of layer: 0
    number of fixed neurons for current layer: 182
    layer progress, group 1 of 25 
        time for training: 167.9241166114807
        actual verification time 0.020189762115478516
        time for verification: 0.05314207077026367
    layer progress, group 2 of 25 
        time for training: 214.1881103515625
        actual verification time 0.010073661804199219
        time for verification: 0.04114532470703125
    layer progress, group 3 of 25 
        time for training: 349.40098094940186
        actual verification time 0.010106086730957031
        time for verification: 0.041626691818237305
    layer progress, group 4 of 25 
        time for training: 228.4954206943512
        actual verification time 0.012079477310180664
        time for verification: 0.05121135711669922
    layer progress, group 5 of 25 
        time for training: 245.0

RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1] because the unspecified dimension size -1 can be any value and is ambiguous

MILP

In [11]:
#print(not_verified_index_snr)
not_verified_index_snr = [ (25, 24), (38, 37), (48, 47), (30, 29),  (19, 19), (22, 22), (26, 25), (29, 28), (32, 31), (33, 32), (42, 41), (43, 42), (46, 45), ]

In [ ]:
group_size = 30
sample_count = 1000
sampling_method = "per_group_sampling"
net_size = [5, 1]
icnn_factory = ICNNFactory("logical", net_size, always_use_logical_layer=False)
#icnn_factory = ICNNFactory("standard", net_size, adapt_layer_for_init=True)
#icnn_factory = ICNNFactory("approx_max", net_size, maximum_function="SMU", function_parameter=0.3)

dhov_verifier_list = []
not_verified_index_dhov = []
sample_index = 0
round_index = 0
samples_to_verify_dhov = len(not_verified_index_snr)
for touple in not_verified_index_snr:
    sample_index = touple[0]
    round_index = touple[1]
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)

    simple_bounds_affine_out, simple_bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])

    print(
        "{}/{} | image index {} ===========================================================================================".format(
            round_index, samples_to_verify_dhov - 1, sample_index))

    overall_time = time.time()

    print("do snr")
    snv_verifier = SingleNeuronVerifier(nn, torch_image, eps, print_log=False, print_new_bounds=False, optimize_bounds=True)
    snv_verifier.generate_constraints_for_net()
    snv_model = snv_verifier.model.copy()
    snv_model.update()
    snv_out_vars = get_output_vars_snv_milp(snv_model, output_size, number_layer * 2)
    add_output_constraints(snv_model, simple_bounds_layer_out, label, snv_out_vars, sovler_bound=None)
    successful_ver = optimize_model(snv_model, snv_out_vars)
    print("snr done")
    bounds_affine_out_snr, bounds_layer_out_snr = snv_verifier.bounds_affine_out, snv_verifier.bounds_layer_out
    dhov_verifier = multidhov.MultiDHOV()
    dhov_verifier.start_verification(nn, torch_image, icnn_factory, group_size, eps=eps, icnn_epochs=200,
                                     init_affine_bounds=bounds_affine_out_snr, init_layer_bounds=bounds_layer_out_snr,
                                     icnn_batch_size=10000, sample_count=sample_count, sample_new=True,
                                     use_over_approximation=True, break_after=None,
                                     sample_over_input_space=False, sample_over_output_space=True, tighten_bounds=False,
                                     use_fixed_neurons_in_grouping=False, sampling_method=sampling_method,
                                     layers_as_snr=[], layers_as_milp=[0,1,2,3],
                                     force_inclusion_steps=3, preemptive_stop=True, even_gradient_training=False,
                                     keep_ambient_space=False, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=False, print_new_bounds=False,
                                     grouping_method="consecutive", group_num_multiplier=5, store_samples=False,
                                     print_optimization_steps=False, print_last_loss=False,
                                     should_plot="none", optimizer="SdLBFGS", init_network=True,
                                     adapt_lambda="included")

    dhov_verifier_list.append(dhov_verifier)

    dhov_model = dhov_verifier.nn_encoding_model.copy()
    dhov_model.update()
    dhov_out_vars = get_output_vars_dhov(dhov_model, output_size, number_layer)
    add_output_constraints(dhov_model, dhov_verifier.bounds_layer_out, label, dhov_out_vars, sovler_bound=float("inf"))
    timeout = 3600 - (time.time() - overall_time)
    t = Timer(timeout, dhov_model.terminate)
    t.start()
    successful_ver = optimize_model(dhov_model, dhov_out_vars)
    t.cancel()
    if not successful_ver:
        not_verified_index_dhov.append(sample_index)
    print("overall time: {}".format(time.time() - overall_time))
    round_index += 1
    sample_index += 1

24/12 | image index 25 ===========================================================================================
do snr
var 0: -1.1911983334571694
var 1: 3.696014224449174
var 2: 10.552146700036063
var 3: 10.628281215586423
var 4: -11.876277338223627
var 5: -0.4592007973448169
var 6: -7.497351074650863
var 7: -1.1693919318586428
var 8: 5.660478473773565
var 9: -4.416768246007123
property NOT verified with max difference 0.07613451555035944
snr done

approximation of layer: 0
    number of fixed neurons for current layer: 183
    encode layer 0 as MILP


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\DHOV\MultiDHOV.py:95: UserWarning: value for group number multiplier is given with grouping method consecutive. consecutive grouping does not use variable number of groups
  warnings.warn("value for group number multiplier is given with grouping method consecutive. "



approximation of layer: 1
    number of fixed neurons for current layer: 209
    encode layer 1 as MILP

approximation of layer: 2
    number of fixed neurons for current layer: 217
    encode layer 2 as MILP

approximation of layer: 3
    number of fixed neurons for current layer: 233
    encode layer 3 as MILP
time for icnn_bound calculation (last layer):0.012115955352783203
done...
var 0: -0.8490433322568531
var 1: 3.7257695927332977
var 2: 11.263598551229084
var 3: 9.861367741343242
var 4: -11.986393031358237
var 5: -0.7215468309916897
var 6: -6.843967085656914
var 7: -0.9657498189235003
var 8: 5.436410020347891
var 9: -5.06364956158648
property verified with max difference -1.402230809885841
overall time: 461.5539035797119
37/12 | image index 38 ===========================================================================================
do snr
var 0: -2.5522522698209027
var 1: 0.4782263827262909
var 2: 8.62400784690656
var 3: 3.7292728479181303
var 4: -1.9951894031055009
var 5: -4